# Merging of ``DataFrames``

## Merging

In [1]:
from pandas import DataFrame, concat
from viedoors import CADLoader, NPALoader, FileMerger, HMLoader, count_duplicates
from viedoors import BSTLoader, FLTLoader, FMLoader, eliminate_duplicates

In [2]:
obj = "420"

cad = CADLoader(file=f"data/{obj}/cad.xlsx", title="CAD")
npa = NPALoader(file=f"data/{obj}/npa.xlsx", title="NPA")
hm = HMLoader(file=f"data/{obj}/hm.xls", title="HM")
bst = BSTLoader(file=f"data/{obj}/bst.xlsx", title="BST")
flt = FLTLoader(file=f"data/{obj}/flt.xlsx", title="FLT")
fm = FMLoader()

df_npa = npa.get_data(prefixed=True)
df_cad = cad.get_data(prefixed=True)
df_hm = hm.get_data(prefixed=True)
df_bst = bst.get_data(prefixed=True)
df_flt = flt.get_data(prefixed=True)
df_fm = fm.get_data(prefixed=True)

# Merge Files

In [3]:
merger = FileMerger(files=[df_cad, df_npa, df_hm, df_bst, df_flt, df_fm], how="left")
merge = merger.get_data_merge()

## Consolidation

In [4]:
merge, info = eliminate_duplicates(merge, "CAD___gar_tuernummer_alt", "NPA___alte_tuernummer")

In [5]:
merge, info = eliminate_duplicates(merge, "CAD___gar_tuernummer_alt", "HM___tuer_nr_alt", info)
merge, info = eliminate_duplicates(merge, "CAD___gar_flucht_tuer_nr", "NPA___fluchtwegs_tuer_nr", info)
merge, info = eliminate_duplicates(merge, "NPA___alte_tuernummer", "FM___brandmeldernr", info)

In [6]:
info

{'420 00A0205.01': 2,
 '420 00A0408.01': 1,
 '420 00A1205.02': 2,
 '420 00A2001.02': 2,
 '420 00A2001.03': 2,
 '420 00A2601.02': 3,
 '420 00A2601.03': 2,
 '420 00A3906.01': 3,
 '420 00B1805.01': 3,
 '420 02A0714.01': 1,
 '420 00A2114.01': 1}

In [6]:
print("Merge lines", len(merge))
print("Unique", len(merge.loc[merge["merge"].duplicated()==False]))
print("Duplicates", len(merge.loc[merge["merge"].duplicated()==True]))

Merge lines 433
Unique 411
Duplicates 22


In [7]:
cad_duplicates = count_duplicates(df_cad)
print(cad_duplicates)

        AKS-Nummer  Anzahl Duplikate
0   420 00B1406.01                 4
1   420 00A2001.02                 2
2   420 00A2001.03                 2
3   420 00A3906.01                 2
4   420 00A2601.02                 2
5   420 00A1205.02                 2
6   420 00A2601.03                 2
7   420 00B1805.01                 2
8   420 02A0714.01                 2
9   420 00A0205.01                 2
10  420 00A0408.01                 2


Am Ende der Konsolidierungsphase wird der Merge als Excel-File exportiert. Davor wurden alle Duplikate eliminiert.

In [8]:
merge.to_excel("matching/420_match_file.xlsx")

## Duplicate analysis

In [23]:
dps = []
for dataset in [df_npa, df_hm, df_bst, df_flt, df_fm]:
    name = dataset.columns[0].split("___")[0]+"-Datenfile"

    if name =="FM-Datenfile":
        break

    print("---")
    print(name)

    fm = FileMerger(files=[df_cad, dataset], how="inner")
    nm = fm.find_non_matching_rows()
    nm.to_excel(f"non_matching/CAD_versus_{name}.xlsx")

    dp = count_duplicates(dataset)
    dp.rename(columns={"Anzahl Duplikate": f"Anzahl Duplikate {name}"}, inplace=True)
    dps.append(dp)
    dp.to_excel(f"duplicates/CAD_versus_{name}.xlsx")

---
NPA-Datenfile
---
HM-Datenfile
---
BST-Datenfile
---
FLT-Datenfile


In [24]:
dps

[            AKS-Nummer  Anzahl Duplikate NPA-Datenfile
 0       420 00B1406.01                               4
 1   nan 00Akeine Nr.00                               3
 2   420 02AKeine Nr.00                               3
 3       420 00A2001.03                               2
 4       420 00A3906.01                               2
 5       420 00A2601.03                               2
 6       420 00A2601.02                               2
 7       420 00A2001.02                               2
 8       420 00A1205.02                               2
 9       420 00A0205.01                               2
 10      420 00B1303.02                               2
 11      420 00B1805.01                               2
 12      nan 00A4601.01                               2,
        AKS-Nummer  Anzahl Duplikate HM-Datenfile
 0  420 02A020A.01                              2
 1  420 00A2114.01                              2,
 Empty DataFrame
 Columns: [AKS-Nummer, Anzahl Duplikate BST-Dat

In [29]:
result = dps[0].merge(dps[1], on='AKS-Nummer', how='outer').merge(dps[2], on='AKS-Nummer', how='outer').merge(dps[3], on='AKS-Nummer', how='outer')
result.fillna(1, inplace=True)

In [30]:
result

,AKS-Nummer,Anzahl Duplikate NPA-Datenfile,Anzahl Duplikate HM-Datenfile,Anzahl Duplikate BST-Datenfile,Anzahl Duplikate FLT-Datenfile
0,420 00A0205.01,2.0,1.0,1.0,1.0
1,420 00A1205.02,2.0,1.0,1.0,1.0
2,420 00A2001.02,2.0,1.0,1.0,1.0
3,420 00A2001.03,2.0,1.0,1.0,1.0
4,420 00A2114.01,1.0,2.0,1.0,1.0
5,420 00A2601.02,2.0,1.0,1.0,1.0
6,420 00A2601.03,2.0,1.0,1.0,1.0
7,420 00A3906.01,2.0,1.0,1.0,1.0
8,420 00B1303.02,2.0,1.0,1.0,1.0
9,420 00B1406.01,4.0,1.0,1.0,1.0
